# Creating Custom Instrument Profiles in PyTestLab

This tutorial will guide you through the process of creating a custom instrument profile for PyTestLab. Instrument profiles are YAML files that define the capabilities and command mappings for a specific instrument, allowing PyTestLab to communicate with it effectively.

## 1. The Basic Structure of a Profile YAML File

An instrument profile is a YAML file that contains all the information PyTestLab needs to control an instrument. The top-level keys in the YAML file define the instrument's identity and its various components.

Here is a minimal example of a profile for a fictional power supply:

In [ ]:
%%writefile my_psu_profile.yaml
manufacturer: MyBrand
model: PSU-101
type: PowerSupply

settings:
  voltage:
    query: "VOLT?"
    command: "VOLT {value}"
  current_limit:
    query: "CURR?"
    command: "CURR {value}"

commands:
  enable_output:
    command: "OUTP ON"
  disable_output:
    command: "OUTP OFF"

The main keys are:
- `manufacturer`: The name of the instrument manufacturer.
- `model`: The model number of the instrument.
- `type`: The instrument type (e.g., `PowerSupply`, `Oscilloscope`, `Multimeter`). This determines which base class PyTestLab will use.
- `settings`: Defines the instrument's configurable settings.
- `commands`: Defines the actions the instrument can perform.

## 2. Defining Instrument Settings

The `settings` section maps high-level attribute names (like `voltage`) to the corresponding SCPI commands. Each setting can have a `query` to read the value and a `command` to set it.

In our example:
```yaml
settings:
  voltage:
    query: "VOLT?"
    command: "VOLT {value}"
  current_limit:
    query: "CURR?"
    command: "CURR {value}"
```
When you access `instrument.voltage`, PyTestLab sends the `VOLT?` query. When you set `instrument.voltage = 5.0`, it sends `VOLT 5.0`.

## 3. Defining Commands

The `commands` section defines methods that control the instrument. These are simple, parameter-less actions.

In our example:
```yaml
commands:
  enable_output:
    command: "OUTP ON"
  disable_output:
    command: "OUTP OFF"
```
Calling `instrument.enable_output()` will send the `OUTP ON` command to the instrument.

## 4. Defining Parameters

For more complex interactions that require arguments, you can use the `parameters` section. This is useful for commands that take arguments but don't fit the simple `setting = value` model.

For example, to set the voltage and current on a specific channel:
```yaml
parameters:
  set_channel_config:
    command: "INST:NSEL {channel}; VOLT {voltage}; CURR {current}"
```
You could then call `instrument.set_channel_config(channel=1, voltage=3.3, current=0.5)`.

## 5. Loading and Using the New Profile

Once you have created your profile YAML file, you can load it in PyTestLab and connect to your instrument. You can use a simulated backend for testing without a physical instrument.

In [ ]:
from pytestlab.instruments import PowerSupply
"from pytestlab.instruments.backends.sim_backend import SimBackend
",

# Create a simulated backend
sim_backend = SimBackend()

# Load the instrument using the custom profile
psu = PowerSupply(profile_file="my_psu_profile.yaml", backend=sim_backend)

# Set the voltage and current limit
psu.voltage = 5.0
psu.current_limit = 1.0

# Verify the settings
print(f"Voltage set to: {psu.voltage} V")
print(f"Current limit set to: {psu.current_limit} A")

# Enable the output
psu.enable_output()
print("Output enabled.")

# You can inspect the commands sent to the simulated backend
print("\nCommands sent to the instrument:")
for cmd in sim_backend.log:
    print(cmd)

## Conclusion

You have now learned the basics of creating a custom instrument profile in PyTestLab. By defining the structure, settings, commands, and parameters in a YAML file, you can extend PyTestLab to support a wide range of instruments.

In [ ]:
import os

# Clean up the created profile file
os.remove("my_psu_profile.yaml")